### Understand Data Structure

In [4]:
# for .csv inside ../data/, print each first line
import os
import csv

for filename in os.listdir('../data/'):
    if filename.endswith('.csv'):
        with open('../data/' + filename) as f:
            reader = csv.reader(f)
            print(next(reader))

['app_id', 'title', 'date_release', 'win', 'mac', 'linux', 'rating', 'positive_ratio', 'user_reviews', 'price_final', 'price_original', 'discount', 'steam_deck']
['app_id', 'helpful', 'funny', 'date', 'is_recommended', 'hours', 'user_id', 'review_id']
['user_id', 'products', 'reviews']


In [5]:
# for .json inside ../data/, print sturcture
import json, os

for filename in os.listdir('../data/'):
    if filename.endswith('.json'):
        with open('../data/' + filename, encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    if isinstance(data, dict):
                        print(f"File: {filename}")
                        print("Header (keys):", list(data.keys()))
                        break
                except json.JSONDecodeError:
                    print(f"Error decoding JSON in line: {line}")


File: games_metadata.json
Header (keys): ['app_id', 'description', 'tags']


### Merge Data

In [3]:
import pandas as pd
import json

In [4]:
# load data
games_df = pd.read_csv('../data/games.csv')
users_df = pd.read_csv('../data/users.csv')
recommendations_df = pd.read_csv('../data/recommendations.csv')

games_metadata = []
with open('../data/games_metadata.json', 'r') as f:
    for line in f:
        games_metadata.append(json.loads(line))
games_metadata_df = pd.json_normalize(games_metadata)

In [5]:
# merge game info
merged_games_df = pd.merge(games_df, games_metadata_df, on='app_id', how='inner')
print(merged_games_df.head())

   app_id                              title date_release   win    mac  linux  \
0   13500  Prince of Persia: Warrior Within™   2008-11-21  True  False  False   
1   22364            BRINK: Agents of Change   2011-08-03  True  False  False   
2  113020       Monaco: What's Yours Is Mine   2013-04-24  True   True   True   
3  226560                 Escape Dead Island   2014-11-18  True  False  False   
4  249050            Dungeon of the ENDLESS™   2014-10-27  True   True  False   

          rating  positive_ratio  user_reviews  price_final  price_original  \
0  Very Positive              84          2199         9.99            9.99   
1       Positive              85            21         2.99            2.99   
2  Very Positive              92          3722        14.99           14.99   
3          Mixed              61           873        14.99           14.99   
4  Very Positive              88          8784        11.99           11.99   

   discount  steam_deck               

In [7]:
# merge review with user info
merged_review_df = pd.merge(recommendations_df, users_df, on='user_id', how='inner')
# drop review_id column
merged_review_df.drop(columns=['review_id'], inplace=True)
# rename columns
merged_review_df.rename(columns={'reviews': 'user'}, inplace=True)
print(merged_review_df.head())

    app_id  helpful  funny        date  is_recommended  hours  user_id  \
0   975370        0      0  2022-12-12            True   36.3    51580   
1   304390        4      0  2017-02-17           False   11.5     2586   
2  1085660        2      0  2019-11-17            True  336.5   253880   
3   703080        0      0  2022-09-23            True   27.4   259432   
4   526870        0      0  2021-01-10            True    7.9    23869   

   products  reviews  
0        28        5  
1       269        1  
2       237        2  
3         5        2  
4        13        2  
